# Inference with pipeline

In [1]:
from transformers import pipeline

/mnt/custom-file-systems/efs/fs-0252e317d4af1dc34_fsap-0a708b50be80889d5/CLONED_REPOS/LLM-World/llmenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

pipe = pipeline(
    'text-generation', # Tells the pipeline we would like to generate text
    model='openai/gpt-oss-20b', # specifies the model
    torch_dtype='auto', # data type the model uses in PyTorch
    device_map='auto') # maps the model to a device

messages = [{'role':'user', 'content':'Explain what mixture-of-experts is'}]

ouput = pipe(messages,
 max_new_tokens=200, # number of tokens the model will generate (output length)
 temperature=1.0) # 1 = baseline randomness in token selection

Loading checkpoint shards: 100%|██████████| 3/3 [02:05<00:00, 41.92s/it]
Device set to use cuda:0


In [3]:
ouput

[{'generated_text': [{'role': 'user',
    'content': 'Explain what mixture-of-experts is'},
   {'role': 'assistant',
    'content': 'analysisThe user asks: "Explain what mixture-of-experts is". We need to respond with an explanation. Provide a clear definition, context, usage, benefits, drawbacks, and possibly reference applications, especially in machine learning and deep learning.\n\nWe should explain conceptually: mixture of experts (MoE) is a type of ensemble learning model that uses a gating network to choose among several expert models. Provide background, the general architecture, how gating network works, training objectives. Examples: mixture models, mixture of logistic regressions, mixture of neural networks, mixture-of-experts layer in transformers.\n\nMention that MoE is used in large-scale language models for scaling with sparsity, e.g., GShard, Switch Transformers. Also mention that MoE reduces computation by activating only a subset of experts per input.\n\nDiscuss issue

In [5]:
ouput[0].keys()

dict_keys(['generated_text'])

In [6]:
ouput[0]['generated_text']

[{'role': 'user', 'content': 'Explain what mixture-of-experts is'},
 {'role': 'assistant',
  'content': 'analysisThe user asks: "Explain what mixture-of-experts is". We need to respond with an explanation. Provide a clear definition, context, usage, benefits, drawbacks, and possibly reference applications, especially in machine learning and deep learning.\n\nWe should explain conceptually: mixture of experts (MoE) is a type of ensemble learning model that uses a gating network to choose among several expert models. Provide background, the general architecture, how gating network works, training objectives. Examples: mixture models, mixture of logistic regressions, mixture of neural networks, mixture-of-experts layer in transformers.\n\nMention that MoE is used in large-scale language models for scaling with sparsity, e.g., GShard, Switch Transformers. Also mention that MoE reduces computation by activating only a subset of experts per input.\n\nDiscuss issues: load balancing, training 

In [8]:
ouput[0]['generated_text'][0].keys()

dict_keys(['role', 'content'])

In [19]:
print(ouput[0]['generated_text'][0]['role'])
print(ouput[0]['generated_text'][0]['content'])


user
Explain what mixture-of-experts is


In [21]:
print(ouput[0]['generated_text'][1]['role'])
print(ouput[0]['generated_text'][1]['content'])


assistant
analysisThe user asks: "Explain what mixture-of-experts is". We need to respond with an explanation. Provide a clear definition, context, usage, benefits, drawbacks, and possibly reference applications, especially in machine learning and deep learning.

We should explain conceptually: mixture of experts (MoE) is a type of ensemble learning model that uses a gating network to choose among several expert models. Provide background, the general architecture, how gating network works, training objectives. Examples: mixture models, mixture of logistic regressions, mixture of neural networks, mixture-of-experts layer in transformers.

Mention that MoE is used in large-scale language models for scaling with sparsity, e.g., GShard, Switch Transformers. Also mention that MoE reduces computation by activating only a subset of experts per input.

Discuss issues: load balancing, training inefficiencies, memory. Mention training strategies: token-based gating, update on active experts onl